In [1]:
!nvidia-smi

Tue May  6 06:56:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-im4tqw99
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-im4tqw99
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-7n3w01uy/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp92p56qlt".


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
!apt-get update -qq
!apt-get install -y -qq cuda-nvcc-11-8

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package cuda-cccl-11-8.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../0-cuda-cccl-11-8_11.8.89-1_amd64.deb ...
Unpacking cuda-cccl-11-8 (11.8.89-1) ...
Selecting previously unselected package cuda-toolkit-11-config-common.
Preparing to unpack .../1-cuda-toolkit-11-config-common_11.8.89-1_all.deb ...
Unpacking cuda-toolkit-11-config-common (11.8.89-1) ...
Selecting previously unselected package cuda-toolkit-11-8-config-common.
Preparing to unpack .../2-cuda-toolkit-11-8-config-common_11.8.89-1_all.deb ...
Unpacking cuda-toolkit-11-8-config-common (11.8.89-1) ...
Selecting previously unselected package cuda-cudart-11-8.
Preparing to unpack .../3-cuda-cudart-11-8_11.8.89-1_amd64.deb ...
Unpacking cuda-cudart-11-8 (11.8.89

In [8]:
%%writefile somethingsomething.cu
// Includes we need: CUDA stuff, random number gen, and basic types
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <curand_kernel.h>
#include <stdint.h> // for uint64_t
#include <time.h>   // for seeding RNGs

// This is just some handy typedefs
typedef unsigned long long uint64;
typedef unsigned __int128 uint128; // works only on GCC/Clang

// Holds parameters we need for Montgomery arithmetic
typedef struct {
    uint64 n;
    uint64 n_prime;         // n' so that n * n' = -1 mod 2^64
    uint64 r_squared_mod_n; // R^2 mod n (R = 2^64)
} MontgomeryParams;

// --- Montgomery Arithmetic Stuff ---

// Figure out n' such that n * n' ≡ -1 mod 2^64
// Uses Newton’s method, because it's quick and does the trick apparently
__device__ uint64 compute_n_prime(uint64 n) {
    uint64 n_prime = n;
    for (int i = 0; i < 5; ++i) {
        n_prime *= (2ULL - n * n_prime);
    }
    return -n_prime;
}

// Basic (a * b) mod n without any fancy optimization
__device__ uint64 mul_mod(uint64 a, uint64 b, uint64 n) {
    uint128 product = (uint128)a * b;
    return (uint64)(product % n);
}

// Get R^2 mod n where R = 2^64 — needed for Montgomery magic
__device__ uint64 compute_r_squared(uint64 n) {
    uint128 r = 1;
    r <<= 64; // Now r = 2^64
    uint64 r_mod_n = (uint64)(r % n);
    uint128 r_mod_n_sq = (uint128)r_mod_n * r_mod_n;
    return (uint64)(r_mod_n_sq % n);
}

// Sets up Montgomery parameters for a given modulus n
__device__ void compute_montgomery_params(uint64 n, MontgomeryParams* params) {
    params->n = n;
    params->n_prime = compute_n_prime(n);
    params->r_squared_mod_n = compute_r_squared(n);
}

// Do a Montgomery reduction: squish T down by R mod n
__device__ uint64 montgomery_reduce(uint128 t, const MontgomeryParams* params) {
    uint64 n = params->n;
    uint64 n_prime = params->n_prime;

    uint64 m = (uint64)t * n_prime; // Multiply low 64 bits by n'

    uint128 sum = t + (uint128)m * n;
    uint64 result = (uint64)(sum >> 64); // Divide by R = 2^64 via bitshift

    // Make sure result is within [0, n)
    if (result >= n) {
        result -= n;
    }
    return result;
}

// Montgomery multiplication: a * b * R^-1 mod n
__device__ uint64 montgomery_multiply(uint64 a_mont, uint64 b_mont, const MontgomeryParams* params) {
    uint128 product = (uint128)a_mont * b_mont;
    return montgomery_reduce(product, params);
}

// Convert a normal number to Montgomery form
__device__ uint64 to_montgomery(uint64 a, const MontgomeryParams* params) {
    return montgomery_multiply(a, params->r_squared_mod_n, params);
}

// Convert from Montgomery form back to regular integer
__device__ uint64 from_montgomery(uint64 a_mont, const MontgomeryParams* params) {
    return montgomery_reduce((uint128)a_mont, params);
}

// --- Modular Exponentiation with Montgomery ---

// Compute base^exponent mod n using Montgomery methods
__device__ uint64 mod_pow_montgomery(uint64 base, uint64 exponent, const MontgomeryParams* params) {
    uint64 n = params->n;
    if (n == 1) return 0; // Trivial edge case

    // Work in Montgomery space
    uint64 base_mont = to_montgomery(base % n, params);
    uint64 result_mont = to_montgomery(1, params); // Start with 1

    while (exponent > 0) {
        if (exponent & 1) {
            result_mont = montgomery_multiply(result_mont, base_mont, params);
        }
        base_mont = montgomery_multiply(base_mont, base_mont, params);
        exponent >>= 1;
    }

    return from_montgomery(result_mont, params);
}

// --- Rabin-Miller Primality Test ---

// GPU-side Rabin-Miller test — checks if n is probably prime
__device__ int rabin_miller_test_device(uint64 n, int k, curandState* rand_state, MontgomeryParams* params) {
    // Quick checks for small numbers and even numbers
    if (n <= 1 || n == 4) return 0;
    if (n <= 3) return 1;
    if ((n & 1) == 0) return 0;

    compute_montgomery_params(n, params);

    // Decompose n - 1 as d * 2^s
    uint64 d = n - 1;
    int s = 0;
    while ((d & 1) == 0) {
        d >>= 1;
        s++;
    }

    for (int i = 0; i < k; ++i) {
        uint64 a;
        do {
            double rand_double = curand_uniform_double(rand_state);
            a = (n > 4) ? 2ULL + (uint64)(rand_double * (double)(n - 3ULL)) : 2ULL;
        } while (a < 2 || a >= n - 1);

        uint64 x = mod_pow_montgomery(a, d, params);

        if (x == 1 || x == n - 1) continue;

        int r;
        for (r = 0; r < s - 1; ++r) {
            uint64 x_mont = to_montgomery(x, params);
            x = from_montgomery(montgomery_multiply(x_mont, x_mont, params), params);

            if (x == n - 1) break;
            if (x == 1) return 0;
        }

        if (r == s - 1 && x != n - 1) return 0;
    }

    return 1; // Probably prime!
}

// --- CUDA Kernels ---

// Set up CURAND RNG states (one per thread)
__global__ void init_curand_kernel(curandState *states, int num_elements, unsigned long long seed) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < num_elements) {
        curand_init(seed, tid, 0, &states[tid]); // Unique sequence per thread
    }
}

// Kernel to run Rabin-Miller on an array of numbers
__global__ void rabin_miller_kernel(uint64* n_values, int* results, int num_elements, curandState* rand_states) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < num_elements) {
        uint64 n = n_values[tid];
        curandState local_rand_state = rand_states[tid]; // Copy state locally
        MontgomeryParams params;

        results[tid] = rabin_miller_test_device(n, K_WITNESSES, &local_rand_state, &params);

        rand_states[tid] = local_rand_state; // Don't forget to write it back!
    }
}


// --- Helper to check CUDA errors ---
void checkCuda(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Error: %s (%s)\n", msg, cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

// --- Main function ---
int main() {
    // Example numbers to test
    uint64 h_n[] = {
        2, 3, 4, 5, 7, 11, 13, 17, 19, 23, // Small primes/composites
        100, 101, 103, 107, 109,          // Around 100
        999999999989ULL,                  // Large prime
        1000000000000000003ULL,           // Larger prime
        1000000000000000005ULL,           // Composite (ends in 5)
        1000000000000000007ULL,           // Prime
        (1ULL << 61) - 1,                 // Mersenne prime M61
        ((1ULL << 61) - 1) * 2,           // Even composite
        ((1ULL << 61) - 1) * 3,           // Composite
        1152921504606846899ULL,           // Prime ( near 2^60 )
        1152921504606846901ULL,           // Composite
         7919, 137, 65537,                 // Primes
         1729, 99, 561                     // Composites (561 = Carmichael)
    };
    int num_elements = sizeof(h_n) / sizeof(h_n[0]);
    int* h_results = (int*)malloc(num_elements * sizeof(int));

    if (!h_results) {
        fprintf(stderr, "Failed to allocate host memory for results.\n");
        return 1;
    }

    printf("Testing %d numbers for primality using CUDA Rabin-Miller with Montgomery Multiplication...\n", num_elements);

    // Allocate device memory
    uint64* d_n;
    int* d_results;
    curandState* d_rand_states;
    checkCuda(cudaMalloc(&d_n, num_elements * sizeof(uint64)), "cudaMalloc d_n");
    checkCuda(cudaMalloc(&d_results, num_elements * sizeof(int)), "cudaMalloc d_results");
    checkCuda(cudaMalloc(&d_rand_states, num_elements * sizeof(curandState)), "cudaMalloc d_rand_states");

    // Copy input numbers to device
    checkCuda(cudaMemcpy(d_n, h_n, num_elements * sizeof(uint64), cudaMemcpyHostToDevice), "cudaMemcpy h_n to d_n");

    // Initialize CURAND states on the device
    int threads_per_block_init = 256;
    int blocks_init = (num_elements + threads_per_block_init - 1) / threads_per_block_init;

    init_curand_kernel<<<blocks_init, threads_per_block_init>>>(d_rand_states, num_elements, time(NULL));
    checkCuda(cudaGetLastError(), "Kernel launch failure: init_curand_kernel");
    checkCuda(cudaDeviceSynchronize(), "cudaDeviceSynchronize after init_curand");

    // Configure and launch the Rabin-Miller kernel
    int threads_per_block = 256;
    int blocks = (num_elements + threads_per_block - 1) / threads_per_block;
    printf("Launching kernel with %d blocks, %d threads per block.\n", blocks, threads_per_block);

    rabin_miller_kernel<<<blocks, threads_per_block>>>(d_n, d_results, num_elements, d_rand_states);
    checkCuda(cudaGetLastError(), "Kernel launch failure: rabin_miller_kernel");

    // Synchronize to ensure kernel completion before copying results
    checkCuda(cudaDeviceSynchronize(), "cudaDeviceSynchronize after kernel");

    // Copy results back to host
    checkCuda(cudaMemcpy(h_results, d_results, num_elements * sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy d_results to h_results");

    // Print results
    printf("Results (%d witnesses):\n", K_WITNESSES);
    for (int i = 0; i < num_elements; ++i) {
        printf("  %llu: %s\n", h_n[i], h_results[i] ? "Probably Prime" : "Composite");
    }

    // Free memory
    free(h_results);
    checkCuda(cudaFree(d_n), "cudaFree d_n");
    checkCuda(cudaFree(d_results), "cudaFree d_results");
    checkCuda(cudaFree(d_rand_states), "cudaFree d_rand_states");

    printf("Done.\n");
    return 0;
}

Overwriting somethingsomething.cu


In [9]:
!nvcc -arch=sm_75 somethingsomething.cu -o rabin_miller

In [10]:
!time ./rabin_miller

Testing 30 numbers for primality using CUDA Rabin-Miller with Montgomery Multiplication...
Launching kernel with 1 blocks, 256 threads per block.
Results (10 witnesses):
  2: Probably Prime
  3: Probably Prime
  4: Composite
  5: Probably Prime
  7: Probably Prime
  11: Probably Prime
  13: Probably Prime
  17: Probably Prime
  19: Probably Prime
  23: Probably Prime
  100: Composite
  101: Probably Prime
  103: Probably Prime
  107: Probably Prime
  109: Probably Prime
  999999999989: Probably Prime
  1000000000000000003: Probably Prime
  1000000000000000005: Composite
  1000000000000000007: Composite
  2305843009213693951: Probably Prime
  4611686018427387902: Composite
  6917529027641081853: Composite
  1152921504606846899: Composite
  1152921504606846901: Composite
  7919: Probably Prime
  137: Probably Prime
  65537: Probably Prime
  1729: Composite
  99: Composite
  561: Composite
Done.

real	0m0.177s
user	0m0.016s
sys	0m0.113s
